Adapted from example here: https://colab.research.google.com/drive/1vIjBtePIZwUaHWfjfNHzBjwuXOyU_ugD?usp=sharing

In [1]:
# pip installs
!pip install -q -U peft transformers datasets bitsandbytes trl==0.19.1 accelerate wandb

In [2]:
# huggingface login
from huggingface_hub import notebook_login

notebook_login()

In [3]:
# load model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig
from google.colab import userdata

# Load the r32 peft model
model_id = "erica-landreth/Meta-Llama-3-8B-Instruct-qlora-medical-r32"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    token=userdata.get('notebook2'))

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
# Load tokenizer
base_model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
# Set it to a new token to correctly attend to EOS tokens.
tokenizer.add_special_tokens({'pad_token': '<PAD>'})

1

In [5]:
# prepare dataset
from datasets import load_dataset

train_dataset = load_dataset("erica-landreth/r32-bootstrapping", split="train",
                             token=userdata.get('notebook2'))

# ds = pd.DataFrame(ds["train"])
# ds = ds[:100]
# ds.head()

# prompt_template = """
# Below is an instruction that describes a task. Write a response that appropriately completes the request.
# ### Instruction:
# You are an experienced medical professional. Answer the following medial question:
# {}
# ### Response:
# <think>
# {}
# </think>
# {}
# """

# ds['text'] = ds.apply(lambda x: prompt_template.format(x['Question'], x['Complex_CoT'], x['Response']), axis=1)
# ds = Dataset.from_pandas(ds[['text']])
# ds = ds.map(lambda x: tokenizer(x['text'], truncation=True), batched=True)

In [6]:
# not sure why???
from huggingface_hub import notebook_login

notebook_login()

In [7]:
model_base = model_id.split("/")[-1]
model_name = f"{model_base}-bootstrap"
YOUR_HF_USERNAME = "erica-landreth"

output_dir = f"{YOUR_HF_USERNAME}/{model_name}"
output_dir

'erica-landreth/Meta-Llama-3-8B-Instruct-qlora-medical-r32-bootstrap'

In [8]:
# define training arguments and trainer
from transformers import TrainingArguments

model_base = model_id.split("/")[-1]
model_name = model_name
YOUR_HF_USERNAME = "erica-landreth"

output_dir = f"{YOUR_HF_USERNAME}/{model_name}"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 300
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
    # push_to_hub=True,
)

In [9]:
train_dataset[0]

{'Question': 'An 11-year-old boy exhibits swearing outbursts, facial contortions, repetitive blinking, and grunting, leading to bullying issues. His father seeks a medication that can help alleviate these symptoms. What medication is most likely to be beneficial for this child?',
 'Answer': "In this case, the medication most likely to be beneficial for the child's behavioral symptoms is a type of antipsychotic, specifically olanzapine. Olanzapine is known for its effectiveness in managing behaviors associated with irritability, aggression, and mood fluctuations, particularly in children with autism. It's also important to consider a thorough evaluation by a healthcare professional to confirm the appropriate diagnosis, as the child's symptoms align with some features of Autism Spectrum Disorder (ASD), which is often associated with these behaviors. Once the diagnosis is confirmed, olanzapine can be considered as a potential medication option to help manage the child's symptoms effective

In [10]:
# define trainer

from trl import SFTTrainer

# https://stackoverflow.com/questions/78688141/how-to-choose-dataset-text-field-in-sfttrainer-hugging-face-for-my-llm-model
def formatting_func(example):
    text = (f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n"
            f"{example['Question']}<|eot_id|><|start_header_id|>"
            f"assistant<|end_header_id|>\n\n{example['Answer']}<|eot_id|>")
    return text

# def formatting_func_general(example,tokenizer):
#     info = {
#         "system": "You are an experienced medical professional.",
#         "user": example['Question'],
#         "assistant": example['Response']
#     }
#     return info

# def formatting_func(example):
#     text = f"""
#     Below is an instruction that describes a task. Write a response that appropriately completes the request.
#     ### Instruction:
#     You are an experienced medical professional. Answer the following medial question:
#     {example['Question']}
#     ### Response:
#     {'Response'}
#     """
#     return text

# def formatting_func(example):
#     text = f"### USER: {example['Question']}\n### ASSISTANT: {example['Response']}"
#     return text

# trainer = SFTTrainer(
#     model=model,
#     args=training_arguments,
#     train_dataset=train_dataset,
#     packing=True,
#     dataset_text_field="id",
#     tokenizer=tokenizer,
#     max_seq_length=1024,
#     formatting_func=formatting_func,
# )

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    # packing=True,
    # dataset_text_field="id",
    # tokenizer=tokenizer,
    # max_seq_length=1024,
    formatting_func=formatting_func,
)

Applying formatting function to train dataset:   0%|          | 0/101 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/101 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/101 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/101 [00:00<?, ? examples/s]

In [11]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: erica-landreth (erica-landreth-university-of-california-berkeley) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,1.195200
20,1.168700
30,1.182800
40,1.160900
50,1.194100
60,1.177900
70,1.192100
80,1.190400
90,1.199600
100,1.164300


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


TrainOutput(global_step=300, training_loss=1.1824779065450033, metrics={'train_runtime': 5149.0873, 'train_samples_per_second': 0.932, 'train_steps_per_second': 0.058, 'total_flos': 6.454137839409562e+16, 'train_loss': 1.1824779065450033})

In [ ]:
# huggingface login
from huggingface_hub import notebook_login

notebook_login()

In [12]:
model.push_to_hub(model_name,token=userdata.get('notebook2'))

Uploading...:   0%|          | 0.00/336M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/erica-landreth/Meta-Llama-3-8B-Instruct-qlora-medical-r32-bootstrap/commit/e0f6cec31a2678f25ac81e5616aa87a92953f0c2', commit_message='Upload LlamaForCausalLM', commit_description='', oid='e0f6cec31a2678f25ac81e5616aa87a92953f0c2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/erica-landreth/Meta-Llama-3-8B-Instruct-qlora-medical-r32-bootstrap', endpoint='https://huggingface.co', repo_type='model', repo_id='erica-landreth/Meta-Llama-3-8B-Instruct-qlora-medical-r32-bootstrap'), pr_revision=None, pr_num=None)

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
out_dir = 'drive/MyDrive/w266_project/'+model_name
out_dir

In [ ]:
trainer.save_model(out_dir)